# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Прочитаем файл /datasets/data.csv и сохраним его в переменной borrowers.
Для наглядности отобразим первые 10 верхних строк таблицы.

Рассмотрим полученную информацию подробнее.
Столбцы содержат следующующие данные:

•	children — количество детей в семье

•	days_employed — общий трудовой стаж в днях

•	dob_years — возраст клиента в годах

•	education — уровень образования клиента

•	education_id — идентификатор уровня образования

•	family_status — семейное положение

•	family_status_id — идентификатор семейного положения

•	gender — пол клиента

•	income_type — тип занятости

•	debt — имел ли задолженность по возврату кредитов

•	total_income — ежемесячный доход
    
•	purpose — цель получения кредита

Для того, чтобы проанализировать качество данных, запрашиваем информацию

In [1]:
import pandas as pd
try:
    borrowers = pd.read_csv('/datasets/data.csv')
except: 
    borrowers = pd.read_csv('data.csv')
display(borrowers.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Видим, что нужно обработать данные, поскольку есть столбцы с отрицательными значениями, например, days_employed. Отрицательные значения, вероятно, возникли из-за совмещения данных из нескольких источников.
Необходимо привести чиловые данные к целочисленным.

In [2]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения. Видно, что строк с данными в разделах days_employed и total_income меньше, чем в остальных столбцах. Необходимо будет заполнить пропуски.


In [3]:
borrowers.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Интересно посмотреть информацию по столбцам, какие категории данных в каждом столбце содержатся. Таким образом, станет очевиднее, какие данные следует очистить.

In [4]:
print(borrowers['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Видно, что нужно будет обработать наименования в столбце education, привести к нижнему регистру

In [5]:
print(borrowers['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В столбце children есть отрицательные значения, нужно это поправить

In [6]:
print(borrowers['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Чтобы систематизировать данные в зависимости от целей кредита, нужно применить лемматизацию, выделив основные цели кредита


In [7]:
print(borrowers['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


Любопытно, что двум безработным дали кредит

In [8]:
print(borrowers['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Обнаружились три типа гендера

In [9]:
print(borrowers['family_status'].value_counts())
print(borrowers['family_status_id'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


Количество соотвествующих позиций по family_status_id и family_status одинаково, очищать и править данные не нужно

In [10]:
borrowers.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Всего в таблице 12 столбцов, тип данных у столбцов int64 и float64, что не вполне удобно, следует привести их к занчениям int.

**Вывод**


Мы изучаем зависимость погашения кредита в срок от семейного положения и количества детей.  Для поверки требуемых основных гипотез понадобятся данные из столбцов debt, family_status и children.

Поскольку столбцы days_employed и total_income, в которых есть пропущенные данные — не ключевые по основной теме данного исследования, то пропуски в этих данных можно будет заполнить и это не повлияет на выводы.

Если потребуется рассмотреть другие связи, а именно, условие своевременного погашения кредита в зависимости от трудового стажа и ежемесячного дохода, то  строки с пропущенными данными лучше будет удалить.

## Шаг 2. Предобработка данных

### Поиск артефактов и избавление от отрицательных величин

Данные по столбцу children до преобразований:

In [11]:
borrowers['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видно, что попало отрицательное занчение, что повлияет на расчет, нужно избавиться от минуса.

Замечание по данным. Странно, что в данных оказался резкий скачок от 5 к 20 детям в семье, нужно разбираться с источником данных, чтобы исключить ошибки и понять причины такого скачка.
Максимально в семьях по 20 детей, заемщиков с таки количеством детей 76, что приводит к выводу, что это не единичный артефакт. Предположим, что в значение 20 попали семьи, в которых более 5 детей.

In [12]:
#Приведем значение в столбце к значению по модулю
borrowers['children'] = borrowers['children'].abs()

#Теперь разбиение по количеству детей в семье верное
print(borrowers['children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Избавимся от минусов и в столбцах days_employed, total_income

In [13]:
borrowers['days_employed'] = borrowers['days_employed'].abs()
borrowers['total_income'] = borrowers['total_income'].abs()

# проверим новые минимальные значения days_employed после избавления от минусов
borrowers[borrowers['days_employed'] == borrowers['days_employed'].min()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование


In [14]:
# проверим новые минимальные значения total_income после избавления от минусов
borrowers[borrowers['total_income'] == borrowers['total_income'].min()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667.263793,недвижимость


Замечание по данным. 
Как видно, максимальный трудовой стаж составил 359219 дней, если бы человек работал 365 дней в году и начал работать сразу после рожления, то его возраст составлял бы 984 года. Надо проверять исходные данные.

**Вывод**

Проведена замена отрицательных значений в столбцах на равные им по модулю положительные. Выявлены несколько столбцов, требующих уточнения по исходным данным (days_employed, children)

### Обработка пропусков

#### Заполнение пропусков раздела days_employed

Можно предположить, что трудовой стаж в днях days_employed будет зависеть от возраста клиента dob_years, чтобы заполнить пустые строки days_employed наиболее релевантными занчениями. 

Заполним пропуски, применив медианные значения, как более объективные по сравнению со средними.

In [15]:
print('Пропуски до -', borrowers['days_employed'].isna().sum())

#заполняем пропущенные значения в days_employed медианными
borrowers['days_employed'] = borrowers['days_employed'].fillna(borrowers.groupby('dob_years')['days_employed'].transform(lambda x: x.fillna(x.mean())))

print('Пропуски после -', borrowers['days_employed'].isna().sum())

Пропуски до - 2174
Пропуски после - 0


In [16]:
borrowers = borrowers.sort_values(by = 'days_employed', ascending = False)
display(borrowers.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


**Вывод**

Заполнены данные по пустым строкам столбца days_employed

При переходе стажа между 56 и 57 годами, резко возрастает среднее значение трудового стажа в днях. Очевидна ошибка в данных, после 56 лет, данные, вероятно, предоставлены в других единицах изменрения. 

#### Заполнение пропусков раздела total_income

Для заполнения пропусков раздела ежемесячного дохода total_income применим группировку по двум переменным (education_id, gender). 

In [17]:
print('Пропуски до -', borrowers['total_income'].isna().sum())

#фрейм с группированными значениями
medians = (borrowers.groupby(['education_id', 'gender']).agg({'total_income': 'median'}).rename(columns = {'total_income': 'median_total_income'}))

#прикрепляем к исходному файлу
borrowers = borrowers.merge(medians, on = ['education_id', 'gender'])
display(borrowers.head(3))

#заполняем пустые значения total_income на медианные median_total_income
borrowers.loc[borrowers['total_income'].isna(), 'total_income'] = borrowers.loc[borrowers['total_income'].isna(), 'median_total_income']
print('Пропуски после -', borrowers['total_income'].isna().sum())

Пропуски до - 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income
0,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью,125632.935658
1,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем,125632.935658
2,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба,125632.935658


Пропуски после - 0


**Вывод**

Заполнены данные по пустым строкам столбца total_income на медианные значения по двум столбцам education_id, gender

### Замена типа данных

Удобнее работать с целочисленными значениями, поэтому приведем тип данных в столбцах days_employed и total_income к типу int64()

In [18]:
borrowers['days_employed'] = borrowers['days_employed'].astype('int')
borrowers['total_income'] = borrowers['total_income'].astype('int')
borrowers['median_total_income'] = borrowers['median_total_income'].astype('int')
borrowers[['days_employed', 'total_income', 'median_total_income']].head(3)

,days_employed,total_income,median_total_income
0,401755,176278,125632
1,401675,126214,125632
2,401663,48286,125632


**Вывод**

Заменили тип данных в столбцах days_employed и total_income на целочисленный

### Обработка дубликатов

Посмотрим, какие есть уникальные наименования уровня образования клиентов

In [19]:
print(borrowers['education'].unique())

['среднее' 'Среднее' 'СРЕДНЕЕ' 'высшее' 'Высшее' 'ВЫСШЕЕ' 'начальное'
 'НАЧАЛЬНОЕ' 'Начальное' 'неоконченное высшее' 'Неоконченное высшее'
 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'ученая степень' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ']


Приведем к единообразию эти наименования. Сперва приведем слова к нижнему регистру.

In [20]:
borrowers['education'] = borrowers['education'].str.lower()
print(borrowers['education'].unique())

['среднее' 'высшее' 'начальное' 'неоконченное высшее' 'ученая степень']


Теперь можно удалить дубликаты, которые могли появиться из-за разных регистров

In [21]:
# Сперва подсчитаем их количество
print('Количество дубликатов -', borrowers.duplicated().sum())

Количество дубликатов - 71


In [22]:
# Удаляем дубликаты
borrowers = borrowers.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов -', borrowers.duplicated().sum())

Количество дубликатов - 0


**Вывод**

Избавились от повторов, привели к единообразию стобец education

### Лемматизация

Чтобы изучить цели заемщиков, применим лемматизацию.
Посмотрим,сколько всего уникальных целей

In [23]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

print(borrowers['purpose'].unique())

['ремонт жилью' 'операции с жильем' 'свадьба' 'покупка недвижимости'
 'сыграть свадьбу' 'покупка жилья для сдачи'
 'получение дополнительного образования' 'образование'
 'покупка коммерческой недвижимости' 'приобретение автомобиля'
 'покупка жилой недвижимости' 'строительство жилой недвижимости'
 'дополнительное образование' 'на покупку своего автомобиля'
 'на покупку автомобиля' 'автомобили' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'сделка с подержанным автомобилем'
 'на покупку подержанного автомобиля' 'заняться высшим образованием'
 'строительство собственной недвижимости' 'получение образования'
 'покупка жилья для семьи' 'профильное образование' 'покупка своего жилья'
 'сделка с автомобилем' 'покупка жилья' 'высшее образование'
 'недвижимость' 'жилье' 'операции с коммерческой недвижимостью'
 'автомобиль' 'свой автомобиль' 'строительство недвижимости'
 'получение высшего образования' 'на проведение свадьбы'
 'заняться образованием']


Видно, что леммы слов повторяются. Посчитаем, сколько раз употребляются слова

In [24]:
purposes_lemmas = """
['заняться образованием' 'автомобили' 'сыграть свадьбу'
 'приобретение автомобиля' 'покупка жилой недвижимости'
 'сделка с подержанным автомобилем' 'покупка коммерческой недвижимости'
 'операции с коммерческой недвижимостью' 'покупка жилья'
 'дополнительное образование' 'покупка жилья для сдачи'
 'на покупку своего автомобиля' 'ремонт жилью' 'операции с жильем'
 'покупка жилья для семьи' 'получение дополнительного образования'
 'сделка с автомобилем' 'строительство недвижимости' 'жилье'
 'на проведение свадьбы' 'недвижимость' 'автомобиль' 'свадьба'
 'заняться высшим образованием' 'строительство жилой недвижимости'
 'строительство собственной недвижимости' 'профильное образование'
 'покупка недвижимости' 'операции со своей недвижимостью' 'образование'
 'покупка своего жилья' 'свой автомобиль' 'операции с недвижимостью'
 'высшее образование' 'на покупку автомобиля'
 'получение высшего образования' 'на покупку подержанного автомобиля'
 'получение образования']
 """

lemmas = m.lemmatize(purposes_lemmas)
print(Counter(lemmas))

Counter({' ': 59, "' '": 22, "'\n": 15, " '": 15, 'покупка': 10, 'недвижимость': 10, 'образование': 9, 'автомобиль': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'получение': 3, 'строительство': 3, 'высокий': 3, 'заниматься': 2, 'жилой': 2, 'сделка': 2, 'подержать': 2, 'коммерческий': 2, 'дополнительный': 2, 'для': 2, '\n': 1, "['": 1, 'сыграть': 1, 'приобретение': 1, 'сдача': 1, 'ремонт': 1, 'семья': 1, 'проведение': 1, 'собственный': 1, 'профильный': 1, 'со': 1, "']\n": 1, ' \n': 1})


Самые популярные цели кредитов: недвижимость, образование, автомобиль, свадьба.
Напишем функцию, которая будет разделять цели на эти четыре группы. 

In [25]:
#возвращает одну из четырех целей заема средств: недвижимость, образование, автомобиль, свадьба
def purpose_group_loan(row):

    lemm = m.lemmatize(row['purpose'])
    if 'недвижимость' in lemm or 'жилье' in lemm or 'операция' in lemm or 'строительство' in lemm or 'жилой' in lemm:
        return 'недвижимость'
    elif 'автомобиль' in lemm:
        return 'автомобиль'    
    elif 'образование' in lemm:
        return 'образование'
    return 'свадьба'

#распечатаем первые 10 строк для проверки  
borrowers['purpose_group'] = borrowers.apply(purpose_group_loan, axis = 1)
print(borrowers[['purpose', 'purpose_group']].head(10))


                                 purpose purpose_group
0                           ремонт жилью  недвижимость
1                      операции с жильем  недвижимость
2                                свадьба       свадьба
3                   покупка недвижимости  недвижимость
4                        сыграть свадьбу       свадьба
5                покупка жилья для сдачи  недвижимость
6  получение дополнительного образования   образование
7                            образование   образование
8      покупка коммерческой недвижимости  недвижимость
9                приобретение автомобиля    автомобиль


In [26]:
#Заменим цели на группированные цели и подсчитаем их количество.
print('Количество займов по целям:')
print(borrowers['purpose_group'].value_counts())

Количество займов по целям:
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64


Видно, что чаще всего кредит берут на недвижимость.

**Вывод**

Основных групп, по которым берут займы, четыре. Из них основная — это операции с недвижимостью. На покупку автомобиля и на образование берут займы примерно в 2,5 раза реже, а на свадьбы - еще реже.

Лемматизация показала, что одно из ключевых слов - "операция", казалось бы, это медицинские цели, но подразумеваются операции с недвижимостью. Стоит избегать таких формулировок для более корректной выдачи результатов. Поскольку уникальных значений не так много по сравнению с количеством строк, процесс заполнения, вероятно, автоматизирован. Поэтому как рекомендация отделу, контролирующему этот процесс, заменить "операции с жильем" на "покупку жилья", например.

### Категоризация данных

Создадим словарь, ставящий в соответствие family_status и family_status_id

In [27]:
borrowers_family_dict = borrowers[['family_status', 'family_status_id']]
borrowers_family_dict = borrowers_family_dict.drop_duplicates().reset_index(drop = True)
borrowers_family_dict.sort_values('family_status_id')

,family_status,family_status_id
1,женат / замужем,0
2,гражданский брак,1
0,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Напишем функцию, которая на вход принимает доход заемщика и
распределяет его по группам по уровням дохода.

In [28]:

def income_group(total_income):
    """
    Возвращает  группу доходов по значению возраста total_income, используя правила:
    - 'первая' при значении total_income < 50 000 
    - 'вторая' при значениии total_income более и равно 50 000 и менее и равно 100 000
    - 'третья' более 100 000 и менее и равно 200 000
    - 'четвертая' во всех остальных случаях
    """
   
    if total_income < 50000:
        return 'первая'
    if (total_income >= 50000) & (total_income <= 100000):
        return 'вторая'
    if (total_income > 100000) & (total_income <= 200000):
        return 'третья'
    return 'четвертая'


Применим фунцию, создав столбец total_income_group, 
который распределяет по четырем категориям клиентов 

In [29]:
borrowers['total_income_group'] = borrowers['total_income'].apply(income_group)
print(borrowers['total_income_group'].value_counts())

третья       11756
четвертая     5235
вторая        4091
первая         372
Name: total_income_group, dtype: int64


Рассмотрим категориальные очищенные данные подробнее.

In [30]:
total_bor = borrowers['education'].count()
print('Всего заемщиков -', total_bor)

Всего заемщиков - 21454


In [31]:
borrowers['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Большая часть заемщиков имеет среднее образование

In [32]:
print(borrowers['gender'].value_counts())

F      14174
M       7279
XNA        1
Name: gender, dtype: int64


Большая часть заемщиков - женщины

In [33]:
print(borrowers['income_type'].value_counts())

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


Чаще всего кредиты берут сотрудники компаний

**Вывод**

Процесс категоризации позволяет выделить в датафрейме определенный сегмент данных, разбить данные по категориям. Создав столбец total_income_group, сможем разделить клиентов на четыре категории по уровню достатка, чтобы проанализировать, какая категория имеет больше долгов.

Словарь family_status family_status_id позволит обращаться не к строке, а к ее численному обозначению, что позволит исключить опечатки при обращении к столбцу family_status.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

По заданию проекта, нам интересны следующие категории:
    количество детей children,   
    семейное положение family_status и family_status_id, 
    есть ли задолженность debt

Создадим сводную таблицу

In [34]:
borrowers_research = borrowers[['children', 'family_status','debt']]
borrowers_pivot = borrowers_research.pivot_table(index=['family_status'], columns = 'children',  aggfunc='sum')
borrowers_pivot.head(10)

debt                                   
children                  0      1      2     3    4    5    20
family_status                                                  
Не женат / не замужем  210.0   52.0    9.0   1.0  1.0  NaN  1.0
в разводе               55.0   21.0    7.0   1.0  0.0  NaN  1.0
вдовец / вдова          53.0    7.0    3.0   0.0  0.0  NaN  0.0
гражданский брак       229.0  118.0   30.0   8.0  0.0  0.0  3.0
женат / замужем        516.0  247.0  145.0  17.0  3.0  0.0  3.0

Таблица показывает данные в абсолютных значениях, видно, что чаще кредит просрочили женатые пары без детей. Интересно, что один водовец/вдова с 20 детьми ни разу кредит не просрочил(а). 

Теперь нужно понять зависимость количества должников по категориям в процентном соотношении от общего количества должников.

In [35]:
borrowers_research = borrowers[['children', 'family_status','debt']]
borrowers_pivot = borrowers_research.pivot_table(index=['family_status'], columns = 'children',  aggfunc=['mean', 'count'])
borrowers_pivot.style.format({('mean','debt', 0):'{:.2%}',('mean','debt', 1):'{:.2%}',('mean','debt',2):'{:.2%}', ('mean','debt',3):'{:.2%}',('mean','debt',4):'{:.2%}', ('mean','debt',5):'{:.2%}', ('mean','debt',20):'{:.2%}'})

In [36]:
#чтобы понять, какой процент семей с детьми имеет долг,
#поделим значения, получившиеcя в разделе debt на 
#количество семей с определенным количеством детей
borrowers_report_ch = borrowers.groupby('children').agg({'debt': ['mean', 'count']})
borrowers_report_ch.columns = ['% должников', 'Количество заемщиков']
borrowers_report_ch = borrowers_report_ch.sort_values(by = '% должников', ascending = False)
borrowers_report_ch.style.format({'% должников' : '{:.2%}'})

,% должников,Количество заемщиков
children,,
20,10.53%,76
4,9.76%,41
2,9.45%,2052
1,9.17%,4855
3,8.18%,330
0,7.54%,14091
5,0.00%,9


**Вывод**

Наибольший процент должников — с 20 детьми. Здесь стоит проверить исходные данные на предмет ошибки, поскольку нет промежуточных данных между 5 детьми и 20, велика вероятность ошибки.

In [37]:
#подсчитаем количество должников по каждому значению статуса
borrowers_report_st = borrowers.groupby('family_status').agg({'debt': ['mean', 'count']})
borrowers_report_st.columns = ['% должников', 'Количество заемщиков']
borrowers_report_st = borrowers_report_st.sort_values(by = '% должников', ascending = False)
borrowers_report_st.style.format({'% должников' : '{:.2%}'})

,% должников,Количество заемщиков
family_status,,
Не женат / не замужем,9.75%,2810
гражданский брак,9.35%,4151
женат / замужем,7.55%,12339
в разводе,7.11%,1195
вдовец / вдова,6.57%,959


**Вывод**

Неженатые и незамужние чаще всего имеют долг по кредиту. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Применим фунцию, создав столбец total_income_group, который распределяет по четырем категориям клиентов согласно их уровню дохода

In [38]:
#подсчитаем количество должников в зависимости от уровня дохода
borrowers_report_inc = borrowers.groupby('total_income_group').agg({'debt': ['mean', 'count']})
borrowers_report_inc.columns = ['% должников', 'Количество заемщиков']
borrowers_report_inc = borrowers_report_inc.sort_values(by = '% должников', ascending = False)
borrowers_report_inc.style.format({'% должников' : '{:.2%}'})

,% должников,Количество заемщиков
total_income_group,,
третья,8.68%,11756
вторая,8.09%,4091
четвертая,6.99%,5235
первая,6.18%,372


**Вывод**

Люди с наименьшим уровнем достатка оказались самыми благонадежными заемщиками, опередив самых состоятельных. Наименее благонадежные клиенты располагают уровнем дохода от 100 000 до 200 000

- Как разные цели кредита влияют на его возврат в срок?

In [39]:
#подсчитаем количество должников в зависимости от пола и цели кредита
borrowers_group_purp_ge = borrowers.groupby(['purpose_group', 'gender']).agg({'debt':['mean', 'count']})
borrowers_group_purp_ge = borrowers_group_purp_ge.sort_values(by = ('debt',  'mean'), ascending = False)
borrowers_group_purp_ge.style.format({('debt',  'mean'):'{:.2%}'})

**Вывод**

Оказалось, что наибольший процент должников среди мужчин, кто брал кредит на образование или автомобиль. Меньше всего должников, кто брал кредиты на недвижимость. Наиболее добропорядочными плательщиками оказались женщины,которые брали кредит на недвижимость.

## Шаг 4. Общий вывод

Проведено исследование, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Данные, предоставленные для анализа были обработаны, среди них выявлены требующие дополнительного уточнения (days_employed, children). Проведена очистка данных, заполнены пропущенные значения, удалены дубликаты, данные категоризированы.

По результатам анализа, существует 4 основные цели, на которые берут займы. Из них основная — это операции с недвижимостью. На покупку автомобиля и на образование берут займы примерно в 2,5 раза реже, а на свадьбы - еще реже. Большая часть заемщиков — женщины, большая часть имеет среднее образование и являются сотрудниками компаний. 

Люди с наименьшим уровнем достатка оказались самыми благонадежными заемщиками, опередив самых состоятельных. Наименее благонадежные клиенты располагают уровнем дохода от 100 000 до 200 000. 

Наибольший процент должников среди мужчин, кто брал кредит на образование или автомобиль, около 12%. Меньше задолжали те, кто брал кредиты на недвижимость, и среди женщин почти на 3% меньше должников, чем среди мужчин (6,27% и 9,07%). 

По теме исследования: 
Люди без детей имеют меньший процент задолженностей, чем люди с детьми. Неженатые на 2% чаще имеют долг по кредиту, чем женатые.